# **Cleaning master data**

Here I create the master dataset using World Bank APIs, FMI and Economic Complexity data.

In [1]:
import wbgapi as wb # World bank
import pandas as pd
from weo import download, WEO # IMF data
import requests, io
import rdata   # or pyreadr
import os

### Variable selection

In [ ]:

# World Bank
wb_variables = [

#    'NY.GDP.PCAP.CD', # GDP per capita (current US$) # Note: I download it in the IMF request
#    'NY.GNP.ATLS.CD', # GNI, Atlas method (current US$)

    # Resource Intensity
    
    'NY.GDP.TOTL.RT.ZS', # Total natural resources rents (% of GDP)

    'NV.IND.MANF.ZS',        # Manufacturing, value added (% of GDP)
    'NV.IND.TOTL.ZS',        # Industry (including construction), value added (% of GDP)
    'TX.VAL.TECH.MF.ZS',     # High-technology exports (% of manufactured exports)
    'NV.AGR.TOTL.ZS',        # Agriculture, forestry, and fishing, value added (% of GDP)
    'NV.SRV.TOTL.ZS',        # Services, value added (% of GDP)
    'NY.GDP.MINR.RT.ZS',     # Mineral rents (% of GDP)
    'NY.GDP.NGAS.RT.ZS',     # Natural gas rents (% of GDP)
    'NY.GDP.PETR.RT.ZS',      # Oil rents (% of GDP)
    
    'NY.ADJ.SVNG.CD', #  Adjusted savings: total (current US$)
    'NY.ADJ.DRES.GN.ZS', # Adjusted savings: natural resources depletion (% of GNI)    
    'IQ.CPA.HRES.XQ', # CPIA building human resources rating (1=low to 6=high)',
    'IQ.CPA.FINQ.XQ', # 'CPIA quality of budgetary and financial management rating (1=low to 6=high)',
    'IQ.CPA.TRAN.XQ', #'CPIA transparency, accountability and corruption in the public sector rating (1=low to 6=high)',
    'DT.DOD.DIMF.CD' # Use of IMF credit (DOD, current US$)
    ]

# IMF 
imf_variables = [
    ("Gross domestic product per capita, constant prices",
     "Purchasing power parity; 2017 international dollar"),
    
    ("General government revenue",
     "Percent of GDP"),
]

# Economic Complexity
eci_variables = [
    'eci'
 ]

# V Democracy
vdem_variables = [
    # High-Level Democracy Indices
    'v2x_polyarchy', # Electoral democracy index
    'v2x_libdem', # Liberal democracy index
    'v2x_partipdem', # Participatory democracy index
    'v2x_delibdem', # Deliberative democracy index
    'v2x_egaldem', # Egalitarian democracy index
    
    # Mid-level indices: Components of Democracy Indices
    #'v2x_api',
    #'v2x_',
    #'v2x_delibdem',
]

# Penn World Table 
pwt_variables = [
    "country",
    "countrycode",
    "year",
    'hc' # Human capital index
    'cn' # Capital stock (national accounts prices)
    'ctfp' # TFP level (constant national prices)
    'cwtfp' # Welfare-relevant TFP    
]


### World Bank API

In [ ]:
def download_wb_indicators(indicators, start_year, end_year):
    final_rows = []

    # Get all real countries
    economies = [c['id'] for c in wb.economy.list() if not c.get("aggregate", False)]
    
    for indicator in indicators:
        print(f"Downloading {indicator} ...")
        raw = wb.data.fetch(indicator, economy=economies, time=range(start_year, end_year + 1))
        
        for row in raw:
            iso = row.get("economy")
            year = int(row.get("time").replace("YR", ""))
            value = row.get("value")
            
            if iso is None or value is None:
                continue

            final_rows.append({
                "Country Code": iso,
                "Year": year,
                "Variable": indicator,
                "Value": value
            })

    df = pd.DataFrame(final_rows)
    return df

wb_df = download_wb_indicators(wb_variables, start_year=1990, end_year=2024)

In [ ]:
# Get all economies (countries + aggregates)
all_economies = wb.economy.list()

# Filter only real countries (exclude aggregates)
countries = [c for c in all_economies if not c.get("aggregate", False)]

# Build a dataframe
country_names = pd.DataFrame({
    "Country Code": [c["id"] for c in countries],
    "Country Name": [c["value"] for c in countries]
})

country_names

,Country Code,Country Name
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,ALB,Albania
4,AND,Andorra
...,...,...
212,XKX,Kosovo
213,YEM,"Yemen, Rep."
214,ZAF,South Africa
215,ZMB,Zambia


#### IMF

In [ ]:
path, _ = download(2024, "Apr")  # or adjust to whichever vintage you want
w = WEO(path)
frames = []
    
frames = []
for subj, unit in imf_variables:
    df = w.get(subj, unit).reset_index().rename(columns={"index": "COUNTRY"})
    df_long = df.melt(id_vars="COUNTRY", var_name="YEAR", value_name="VALUE")
    df_long["INDICATOR"] = subj
    frames.append(df_long)

imf_df = pd.concat(frames, ignore_index=True)
print(imf_df.head())

# Remode Weo data from code folder
os.remove("weo_2024_1.csv")

imf_df = imf_df.rename(columns={
    'COUNTRY': 'Year',
    'YEAR': 'Country Code',
    'INDICATOR': 'Variable',
    'VALUE': 'Value'
})
imf_df

Already downloaded 2024-Apr WEO dataset at weo_2024_1.csv


WEO_ParsingError: Subject must be one of 
Gross domestic product, constant prices, Gross domestic product, current prices, Gross domestic product, deflator, Gross domestic product per capita, constant prices, Gross domestic product per capita, current prices, Output gap in percent of potential GDP, Gross domestic product based on purchasing-power-parity (PPP) share of world total, Implied PPP conversion rate, Total investment, Gross national savings, Inflation, average consumer prices, Inflation, end of period consumer prices, Volume of imports of goods and services, Volume of Imports of goods, Volume of exports of goods and services, Volume of exports of goods, Unemployment rate, Employment, Population, General government revenue, General government total expenditure, General government net lending/borrowing, General government structural balance, General government primary net lending/borrowing, General government net debt, General government gross debt, Gross domestic product corres

#### Economic Complexity

In [ ]:
# ECI
eci_df = (
    pd.read_csv('https://raw.githubusercontent.com/AyaanTigdikar/Capstone/refs/heads/main/rawdata/growth_proj_eci_rankings.csv') # Note: If error, update the raw link in here: https://github.com/AyaanTigdikar/Capstone/blob/main/rawdata/growth_proj_eci_rankings.csv
      .rename(columns={'country_iso3_code': 'country_code', 'eci_hs92': 'eci'})
      .drop(columns=['eci_rank_hs92'])
)

# Country codes
continent_labels = pd.read_csv('../../data/location_group_member.csv')

eci_clean = (
    eci_df
      .merge(
          continent_labels[continent_labels['group_type'] == 'continent'][['group_name', 'country_id']],
          on='country_id',
          how='left'
      )
      .rename(columns={'group_name': 'continent'})
      .drop_duplicates(subset='country_code', keep='first')
)

eci_df['Variable'] = 'Economic Complexity'

eci_df = eci_df.rename(columns={
  'country_code': 'Country Code',
  'year': 'Year',
  'eci': 'Value'
})

eci_df = eci_df[['Country Code', 'Year', 'Variable' , 'Value']]

### V democracy

In [ ]:
# 1. Download the RData from GitHub “raw” URL
url = ("https://raw.githubusercontent.com/vdeminstitute/vdemdata/master/data/vdem.RData")
resp = requests.get(url)
resp.raise_for_status()  # check download succeeded

with open("vdem.RData", "wb") as f:
    f.write(resp.content)

# Load the .RData file
vdem_r = rdata.read_rda("vdem.RData")

# Read R data as a df
vdem = vdem_r.get("vdem")

# Remode RData from code folder
os.remove("vdem.RData")
    
var_list = ['country_name', 'country_text_id', 'year'] + vdem_variables

vdem = vdem[var_list]

vdem = vdem.rename(columns={
                   'country_name': 'Country Name',
                   'country_text_id': 'Country Code',
                   'year': 'Year',
})


vdem_df = pd.melt(
    vdem,
    id_vars=['Country Code','Year'],
    value_vars=['v2x_polyarchy','v2x_libdem', 'v2x_partipdem', 'v2x_delibdem', 'v2x_egaldem'],
    var_name='Variable',
    value_name='Value'
)

vdem_df = vdem_df[vdem_df['Year'] >= 1990]

c:\Users\iorueta\AppData\Local\Programs\Python\Python313\Lib\site-packages\rdata\conversion\_conversion.py:900: UserWarning: Missing constructor for R class "Date". The underlying R object is returned instead.
  warnings.warn(


### Penn World Table

In [ ]:
# Penn World Table 11.0 persistent DOI
doi = "10.34894/FABVLR"
url = f"https://dataverse.nl/api/access/dataset/:persistentId/?persistentId=doi:{doi}"

r = requests.get(url)
with open("pwt110.zip", "wb") as f:
        f.write(r.content)


Downloaded PWT 11.0


## Merging and final cleaning

In [ ]:
final_df = pd.concat([wb_df, eci_df, imf_df, vdem_df])

# Variable names
final_df['Variable'].unique()

rename_map = {
    "NV.IND.MANF.ZS": "Manufacturing",
    "NV.IND.TOTL.ZS": "Industry",
    "TX.VAL.TECH.MF.ZS": "High-tech exports",
    "NV.AGR.TOTL.ZS": "Agriculture",
    "NV.SRV.TOTL.ZS": "Services",
    'NY.GDP.TOTL.RT.ZS': 'Total natural resources rents (% of GDP)', # Total natural resources rents (% of GDP)
    'NY.GDP.MINR.RT.ZS': 'Mineral rents (% of GDP)',     # Mineral rents (% of GDP)
    'NY.GDP.NGAS.RT.ZS': 'Natural gas rents (% of GDP)',     # Natural gas rents (% of GDP)
    'NY.GDP.PETR.RT.ZS': 'Oil rents (% of GDP)',      # Oil rents (% of GDP)
    'NY.ADJ.SVNG.CD': 'Adjusted savings: total (current US$)',
    'NY.ADJ.DRES.GN.ZS': 'Adjusted savings: natural resources depletion (% of GNI)',
    'IQ.CPA.HRES.XQ': 'CPIA building human resources rating (1=low to 6=high)',
    'IQ.CPA.FINQ.XQ': 'CPIA quality of budgetary and financial management rating (1=low to 6=high)',
    'IQ.CPA.TRAN.XQ': 'CPIA transparency, accountability and corruption in the public sector rating (1=low to 6=high)',
    'DT.DOD.DIMF.CD': 'Use of IMF credit (DOD, current US$)',
    
    "Economic Complexity": "Economic Complexity Index",
    "Gross domestic product per capita, constant prices": "GDP per capita (constant prices, PPP)",
    "General government revenue": "Government revenue",

    'v2x_polyarchy': 'electoral_dem', # Electoral democracy index
    'v2x_libdem': 'liberal_dem', # Liberal democracy index
    'v2x_partipdem': 'participatory_dem', # Participatory democracy index
    'v2x_delibdem': 'deliberative_dem', # Deliberative democracy index
    'v2x_egaldem': 'egalitarian_dem' # Egalitarian democracy index
}

final_df["Variable"] = final_df["Variable"].replace(rename_map)

# Countries names
final_df = final_df.merge(country_names, how='left', on='Country Code')

# Filter period
# If Year is a pandas Period, extract the year as int
final_df['Year'] = final_df['Year'].apply(lambda x: x.year if hasattr(x, 'year') else int(x))

final_df = final_df[(final_df['Year'] >= 1990) & (final_df['Year'] <= 2024)]


In [ ]:
final_df

,Country Code,Year,Variable,Value,Country Name
0,ZWE,2021,NY.GDP.TOTL.RT.ZS,6.398452,Zimbabwe
1,ZWE,2020,NY.GDP.TOTL.RT.ZS,4.746668,Zimbabwe
2,ZWE,2019,NY.GDP.TOTL.RT.ZS,4.715765,Zimbabwe
3,ZWE,2018,NY.GDP.TOTL.RT.ZS,3.378189,Zimbabwe
4,ZWE,2017,NY.GDP.TOTL.RT.ZS,6.095448,Zimbabwe
...,...,...,...,...,...
116888,ZZB,2020,egalitarian_dem,0.249000,NaN
116889,ZZB,2021,egalitarian_dem,0.256000,NaN
116890,ZZB,2022,egalitarian_dem,0.265000,NaN
116891,ZZB,2023,egalitarian_dem,0.262000,NaN


#### To wide

In [ ]:
final_df_wide = final_df.pivot(
    index=['Country Code', 'Year'],
    columns='Variable',
    values='Value'
).reset_index()

In [ ]:
final_df_wide

Variable,Country Code,Year,Adjusted savings: natural resources depletion (% of GNI),Adjusted savings: total (current US$),Agriculture,Economic Complexity Index,"GDP per capita (constant prices, PPP)",Government revenue,High-tech exports,Industry,...,Mineral rents (% of GDP),Natural gas rents (% of GDP),Oil rents (% of GDP),Services,Total natural resources rents (% of GDP),deliberative_dem,egalitarian_dem,electoral_dem,liberal_dem,participatory_dem
0,ABW,1990,0.001568,NaN,NaN,NaN,32967.377,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,0.001552,NaN,NaN,NaN,NaN,NaN
1,ABW,1991,0.001649,NaN,NaN,NaN,34292.324,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,0.001634,NaN,NaN,NaN,NaN,NaN
2,ABW,1992,0.001393,NaN,NaN,NaN,35073.111,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,0.001379,NaN,NaN,NaN,NaN,NaN
3,ABW,1993,0.000941,NaN,NaN,NaN,35244.590,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,0.000925,NaN,NaN,NaN,NaN,NaN
4,ABW,1994,0.000953,NaN,NaN,NaN,36215.205,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,0.000937,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7512,ZZB,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.263,0.249,0.271,0.219,0.174
7513,ZZB,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.273,0.256,0.285,0.232,0.180
7514,ZZB,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.278,0.265,0.294,0.240,0.182
7515,ZZB,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.281,0.262,0.298,0.242,0.186


## Saving data

In [ ]:
final_df.to_csv('../workingdata/master_data_long.csv')

final_df_wide.to_csv('../workingdata/master_data_wide.csv')